# 1. SELECT DATASET

In [ ]:
import datasets


In [ ]:
#dataset = datasets.load_from_disk('Z:/Bonz/datasets/bookcorpus')
dataset = datasets.load_dataset('bookcorpusopen', split='train', cache_dir='Z:/Bonz/datasets')

In [6]:
example = dataset[0:2]

In [14]:
def split_text(examples):
    sentences = []
    for example in examples['text']:
        sen = example.split('\n\n')
        sentences += list(filter(lambda s: len(s)>50, sen))
    return {'sentences': sentences}

In [17]:
splitted_dataset = dataset.map(split_text, batched=True, remove_columns=dataset.column_names)

In [34]:
preprocessed_dataset = splitted_dataset.map(lambda s: tokenizer(s['sentences'], padding='max_length', truncation=True), 
                                            batched=True,
                                            batch_size=2**10,
                                            remove_columns=splitted_dataset.column_names,
                                            keep_in_memory=False,
                                            writer_batch_size=2**14,
                                           )

In [46]:
preprocessed_dataset.save_to_disk('Z:/Bonz/datasets/bookcorpus')

In [45]:
preprocessed_dataset

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 25845810
})

In [62]:
import torch

def mask_token(inputs):
    inputs = torch.LongTensor(inputs)
    labels = inputs.clone()

    # Sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
    probability_matrix = torch.full(labels.shape, 0.15)

    # Create special_token matrix
    special_tokens_mask = [tokenizer.get_special_tokens_mask(label, already_has_special_tokens=True) for label in labels.tolist()]
    special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)

    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # Only compute loss on masked tokens

    # Replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 1.0)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    return {'input_ids': inputs.tolist(), 'labels': labels.tolist()}

In [67]:
masked_dataset = preprocessed_dataset.map(lambda x: mask_token(x['input_ids']),
                                          batched=True,
                                          batch_size=2**14,
                                          writer_batch_size=2**16,
                                         )

In [71]:
import time
from torch.utils.data import DataLoader

data_loader = DataLoader(masked_dataset, batch_size=256)
s = time.time()
for a in data_loader:
    print(time.time()-s)
    break

0.4831111431121826


In [74]:
masked_dataset.save_to_disk('Z:/Bonz/datasets/bookcorpus')

# 2. TRAIN A TOKENIZER

In [15]:
from tokenizers.implementations import BertWordPieceTokenizer, ByteLevelBPETokenizer

# Initialize a tokenizer
tokenizer = BertWordPieceTokenizer()

# Customize training
#tokenizer.train(files='bookcorpus.txt', vocab_size=10000, min_frequency=3, special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])


In [19]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', model_max_length=512)

In [30]:
tokenizer.encode('This is Duc', padding='max_length')
tokenizer.convert_ids_to_tokens(tokenizer.encode('What are you playin?', padding='max_length'))

['[CLS]',
 'what',
 'are',
 'you',
 'play',
 '##in',
 '?',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 

In [25]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}